In [1]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
)
from llama_index.embeddings.cohere import CohereEmbedding
from llama_index.llms.cohere import Cohere
from llama_index.core.memory import ChatMemoryBuffer
from IPython.display import Markdown, display

In [2]:
import os
import openai

# needed to synthesize responses later
os.environ["OPENAI_API_KEY"] = "sk-..."
openai.api_key = os.environ["OPENAI_API_KEY"]

In [3]:
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=10)

In [4]:
embed_model = CohereEmbedding(
        cohere_api_key="vORtxj32na8zl2ceIbxH1c5tNziAVWDdAy2x3sbX",
        model_name="embed-english-v3.0", # Supports all Cohere embed models
        input_type="search_query", # Required for v3 models
    )

llm_model = Cohere(
    model='command-r',
    api_key="vORtxj32na8zl2ceIbxH1c5tNziAVWDdAy2x3sbX",
    temperature=0.2,
)

In [5]:
DIR= "/home/surya/NEU/CS5100 FAI/Project/ResearchLens/uploads"

In [ ]:
main_pdf = os.path.join(DIR, "2311.17902.pdf")
citing_pdf = os.path.join(DIR, "2201.02605.pdf")

In [25]:
documents = SimpleDirectoryReader(input_files=[citing_pdf]).load_data()

In [7]:
# index = VectorStoreIndex.from_documents(
#     documents, embed_model=embed_model, transformations=[text_splitter]
# )

In [8]:
import cohere
co = cohere.Client("vORtxj32na8zl2ceIbxH1c5tNziAVWDdAy2x3sbX")

## Feed all documents to the chat

Let's ask something specific from the later pages to verify it is working accurately.

In [10]:
# selected_docs = []

# for doc in documents:
#     if doc.metadata['file_path'] == citing_pdf:
#         selected_docs.append(doc)

Estimate token length

In [11]:
words = 0

for doc in documents:
    words += len(doc.text.split())

words, words * 3/4

(10931, 8198.25)

In [12]:
question = """
Similar to DECOLA Phase 2, we self-
train baseline on weakly-labeled data. For the self-training
algorithm, we use online self-training with max-size loss
from Detic [74] as baseline comparison (baseline + self-
train) to DECOLA Phase 2. We tested max-size and max-
score losses from Detic [74] (online pseudo-labeling) as well
as offline pseudo-labeling similar to DECOLA , and max-size
loss consistently performed the best.

What is the Detic model mentioned in this paragraph? What are the novel contributions of the Detic model?
"""

In [13]:
full_doc_text = "\n".join([doc.text for doc in documents])

In [14]:
prompt = f"""
Given the following question about a paper from the user, please provide a brief compilation of texts from the referring paper that can help answer the question. 

Question: 
{question}

Referring Paper Content:
{full_doc_text}
"""

"Do not rephrase the output text, simply provide the relevant text snippets from the referring paper."

In [15]:
print(prompt)


Given the following question about a paper from the user, please provide a brief compilation of texts from the referring paper that can help answer the question. 

Question: 

Similar to DECOLA Phase 2, we self-
train baseline on weakly-labeled data. For the self-training
algorithm, we use online self-training with max-size loss
from Detic [74] as baseline comparison (baseline + self-
train) to DECOLA Phase 2. We tested max-size and max-
score losses from Detic [74] (online pseudo-labeling) as well
as offline pseudo-labeling similar to DECOLA , and max-size
loss consistently performed the best.

What is the Detic model mentioned in this paragraph? What are the novel contributions of the Detic model?


Referring Paper Content:
Detecting Twenty-thousand Classes
using Image-level Supervision
Xingyi Zhou1,2⋆Rohit Girdhar1Armand Joulin1
Philipp Kr¨ ahenb¨ uhl2Ishan Misra1
1Meta AI2The University of Texas at Austin
Abstract. Current object detectors are limited in vocabulary size due to
the

In [16]:
response = co.chat(
	message=prompt, 
	model="command-r", 
	temperature=0.1,
    max_tokens=1000,
)

answer = response.text

In [17]:
relevant_citing_context = answer

In [18]:
print(answer)

The Detic model mentioned in the paragraph is a simple way to use image supervision in large vocabulary object detection. It trains the classifiers of a detector on image classification data, which enables the detector to identify tens of thousands of concepts. Unlike other models, Detic does not require complex assignment schemes. The novel contributions of the Detic model are as follows:
1. It identifies issues with existing weakly-supervised detection techniques in the open-vocabulary setting and provides a simpler alternative.
2. The model's family of losses significantly improves detection performance on novel classes and matches the supervised performance upper bound.
3. The detector trained with Detic generalizes well to new datasets and vocabularies without finetuning. 
4. The method is simple to implement and ready to use for real-world open-vocabulary detection.


# Answering the question

Original response with no extra-context

In [19]:
response = co.chat(
	message=question, 
	model="command-r", 
	temperature=0.1,
    max_tokens=1000,
)

answer = response.text

In [20]:
print(answer)

The Detic model refers to the work titled "Detic: Online Instance Selection for Weakly Supervised Detection Training" which proposes a novel online instance selection method for weakly-supervised object detection. The method uses a max-size loss function, among other loss functions, to improve the training process. 

The contributions of the Detic model are as follows:
1. Online Instance Selection: The model selects the most informative instances during training by proposing an online instance selection strategy. This strategy helps in focusing on hard, uncertain, and diverse instances, which leads to more effective training.
2. Max-Size Loss Function: Detic introduces the concept of max-size loss, which encourages the detector to assign larger bounding boxes to positive instances. This loss function prioritizes the selection of larger objects, leading to improved detection performance.
3. Efficiency: The online instance selection process is computationally efficient, making it practic

Response with the citing context

In [21]:
new_question = f"""
Citation Context:
{relevant_citing_context}

Question:
{question}
"""

print(new_question)


Citiation Context:
The Detic model mentioned in the paragraph is a simple way to use image supervision in large vocabulary object detection. It trains the classifiers of a detector on image classification data, which enables the detector to identify tens of thousands of concepts. Unlike other models, Detic does not require complex assignment schemes. The novel contributions of the Detic model are as follows:
1. It identifies issues with existing weakly-supervised detection techniques in the open-vocabulary setting and provides a simpler alternative.
2. The model's family of losses significantly improves detection performance on novel classes and matches the supervised performance upper bound.
3. The detector trained with Detic generalizes well to new datasets and vocabularies without finetuning. 
4. The method is simple to implement and ready to use for real-world open-vocabulary detection.

Question:

Similar to DECOLA Phase 2, we self-
train baseline on weakly-labeled data. For the 

In [22]:
response = co.chat(
	message=new_question, 
	model="command-r", 
	temperature=0.1,
    max_tokens=1000,
)

answer = response.text

In [23]:
print(answer)

The Detic model is a novel approach to large vocabulary object detection that uses image supervision. The model's key contributions are:

1. It addresses issues with current open-vocabulary detection techniques by offering a straightforward alternative.
2. The family of losses in the Detic model enhances detection performance on novel classes and reaches the supervised performance upper limit.
3. The Detic model generalizes well to new datasets and vocabs without the need for fine-tuning.
4. The method is simple to implement and use in the real world for open-vocabulary detection.

I hope this helps you understand the Detic model and its contributions. Let me know if you have any other questions!
